# 로이터 뉴스 데이터 분류

In [26]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [27]:
# 데이터 불러오기
from tensorflow.keras.datasets import reuters
# num_words로 사용할 단어 수 지정 가능, None일 경우 데이터셋에 존재하는 모든 단어 사용(데이터 확인용)
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

In [ ]:
# 데이터셋 구조 확인(훈련용 데이터 개수, 테스트용 데이터 개수, 카테고리 수 등)
train_numbers = len(X_train)
test_numbers = len(X_test)
categories = len(np.unique(y_train))

print('훈련용 데이터 개수:', train_numbers)
print('테스트용 데이터 개수:', test_numbers)
print('카테고리 수:', categories)

In [ ]:
# 뉴스 기사의 최대 길이 및 평균길이 출력(그래프 시각화하면 더 좋음)
import matplotlib.pyplot as plt
import numpy as np

len_article = [len(s) for s in X_train]

print('뉴스 기사 최대 길이 : {}'.format(np.max(len_article)))
print('뉴스 기사 평균 길이 : {}'.format(np.mean(len_article)))

plt.subplot(1,2,1)
plt.boxplot(len_article)
plt.subplot(1,2,2)
plt.hist(len_article, bins=50)
plt.show

In [ ]:
# 각 단어와 그 단어에 부여된 인덱스 확인
word_to_index = reuters.get_word_index()
print(word_to_index)

# 로이터 뉴스 분류하기

In [38]:
# num_words, test_split 지정하여 reuters 데이터셋 로드
num_words = 10000
test_split = 0.2
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=num_words, test_split=test_split)

In [39]:
# 95%의 데이터를 포함하는 길이로 maxlen 지정
maxlen = int(np.percentile(lengths, 95))
print('95%: ', maxlen)

# maxlen길이만큼 뉴스 기사 패딩
maxlen = max(lengths)
print('maxlen:', maxlen)

# 레이블 원-핫 인코딩
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [41]:
# 모델 생성
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, SimpleRNN

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=128, input_length=maxlen))
model.add(SimpleRNN(128))
model.add(Flatten())
model.add(Dense(1, activation='softmax'))

model.summary()

In [44]:
# 체크포인트 설정 및 컴파일
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model_news.keras', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
# 모델 학습(history 변수에 학습 과정 저장)
history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2, callbacks=[mc])

In [ ]:
# 정확도 평가(result[0]: 손실, result[1] : 정확도)

result = model.evaluate(X_test, y_test)
print("\n테스트 손실: %.4f" % (result[0]))
print("테스트 정확도: %.4f" % (result[1]))

In [ ]:
# loss 시각화
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# 학습된 가중치 불러와서 평가

In [ ]:
# best_model.keras 파일을 불러와서 테스트 데이터 손실과 정확도를 출력해보세요.
from tensorflow.keras.models import load_model
loaded_model = load_model('best_model_news.keras')

result = loaded_model.evaluate(X_test, y_test)
print("\n테스트 손실: %.4f" % (result[0]))
print("테스트 정확도: %.4f" % (result[1]))